## Introduction to game theory

<small>see sections 3.1, 3.3, 4.1, 4.2, 4.4 and 4.7 of [Multi-Agent Reinforcement Learning: Foundations and Modern Approaches](https://www.marl-book.com) for a more in-depth overview.</small><br>
<small>The game examples and the images are taken from [Game Theory, Alive](https://bookstore.ams.org/mbk-101).</small>

In the previous lesson, we discovered why switching from single agent to multiple agents can be challenging. In this lesson we try to model multi-agent interaction with meaningful theoretical models. We start with **normal form games** which descrive _one-shot_ interactions between multiple agents with no evolving environment and provide the necessary foundations to understand the objective and problems, then we switch to **stochastic games** that model multiple interactions between a group of agents in an evolving environment.

<img src="imgs/game-hierarchy.png" width="600" style="display: block; margin: 0 auto"/>

### Markov decision processes

First recall that a Markov decision process is a tuple composed of
- Finite set of states $\mathcal{S}$
- Finite set of actions $\mathcal{A}$
- Reward function $r: \mathcal{S} \times \mathcal{A} \to \mathbb{R}$
- State transition probability function $\{ p(\cdot \mid s, a) \mid s \in \mathcal{S}, a \in \mathcal{A} \}$ defined on a transition kernel $p(\cdot \mid s, a)$ over $\mathcal{S}$.

### Normal form games

A normal form (or _strategic form_) game is a model that describes the interaction of multiple agents in a static environment. A normal form game is defined as a tuple composed of
- Set of $N$ agents $I = \{1, \dots, N\}$
- For each agent $i \in I$:
	- Finite set of actions $\mathcal{A}_i$
	- Reward function $r_i: \mathcal{A} \to \mathbb{R}$ where $\mathcal{A} := \mathcal{A}_1 \times \dots \times \mathcal{A}_n$

As expected, there is not set of states or a transition function, like in the MDP. This form is called _normal_ in constract to the _extensive_ form, which is more involved and can describe sequential moves, chance moves and even partial information, we won't study extensive form games.

Each player of a normal form game has a policy $\pi_i$ used to describe their action, this policy defines a probability distribution over the actions, such that player $i$ plays action $a_i$ with probability $\pi_i(a_i)$. Each player simoultaneously chooses an action according their probability distribution and the chosen actions together form the _joint action_ $a = (a_1, a_2, \dots, a_N)$. The reward given to each player depends on the joint action $r_i(a)$, not on any player's action alone.

Types of normal-form games include
- **zero-sum games**: a normal form game is called _zero-sum_ if the sum of the rewards perceived by the players is always zero, for example for two players $r_1(a) = -r_2(a)$ for any joint action $a \in \mathcal{A}$.
- **common-reward games**: a normal form game is called _common-reward_ if the same reward is given to every player $r_1(a) = r_2(a) = \dots = r_N(a)$.
- **general-sum games**: a normal form game is called _general-sum_ if no restrictions are placed on the reward function.

We focus on two-player general-sum games. We represent these games as matrices where each row represents the reward of one player and each column represents the rewaard of the other, each cell is a joint action. Note that the only randomness lies in the players' policies.

#### Prisoner's dilemma

<img src="imgs/prisoner-dilemma.jpg" width="400" style="display: block; margin: 0 auto"/>

The prisoner's dilemma is a classic example of general-sum normal form game. Andrea and Bryan are two not very good criminals that get caught and put in separate cells. Fortunately for them, there is not enough evicence for a full conviction. After questioning, the officer separately tells them that they can either **collaborate** with the police by confessing to the crime or remain **silent**. If only one confesses, they get out of prison, but the other is sentenced to 10 years in jail. If both confess, they get 8 years each. If both stay silent, the police has makes a minor conviction of 1 year each.

We can write the game in matrix form as
<center>

|                 | **Stay silent** | **Confess** |
|-----------------|-----------------|-------------|
| **Stay silent** |    $(-1, -1)$   |  $(-10, 0)$ |
| **Confess**     |    $(0, -10)$   |  $(-8, -8)$ |

</center>

The prisoners make their decision separately, what should they do? If they both stay silent they get only one year each, knowing that, one prisoner might try to predict that the other will remain silent and confess, but if the other does the same, they both confess.

<small>This is also true if the game is repeated a fixed number of times between the same two players and they want to maximise their respective cumulative reward. Try to work out why. Hint: start from the last round.</small>

Suppose that one prisoner knows what the other is going to do, if the other is going to stay silent, the best move is to confess, to reduce the sentence to the zero, instead if the other is going to confess, the best move is to confess. In the end the best play seems to be to confess regardless.

Formally, the strategy $\pi_i$ which picks action $a_i = $ _confess_ with probability 1 is **dominant**, meaning that achieves better reward to any other policy regardless of what the other player will do.

This game was studies under the lens of philosophy. From _"The prisoner's dilemma paradox: rationality, morality, and reciprocity"_ by Rory W. Collins:

> Several philosophers have advanced a case for staying silent on rational grounds called the symmetry argument (Bicchieri and Green Reference Bicchieri, Green, Holmström-Hintikka and Tuomela1997). The general idea is that Andrea knows she and Bryan are both rational agents, so whatever choice she arrives at, Bryan will also reach, since two rational agents will reason symmetrically. This reduces the four possibilities to just two: both confess or both stay silent. Of these, the latter is clearly preferable, so Andrea should remain silent, confident that Bryan will too.

Popular mentions of the prisoner's dilemma:
- [It's a Magic: the Gathering card](https://scryfall.com/card/mkc/34/prisoners-dilemma)
- [spoiler alert] One of the last scenes of Christopher Nolan's The Dark Knight sets a scene which mimics the prisoner's dilemma where two boats are set to explode and each has the detonator for the other.

### Stochastic games
We now introduce the notation necessary to formalize the proposed multi-agent systems. Little has changed from the MDPs in RL, we simply need to add multiple players and tweak the reward function.

Consider a multi-agent system with $N > 2$ players identified by their index, call $I = \{ 1, \dots, N \}$ the set of players. The actions $a = (a_1, \dots, a_N)$ are composed of $N$ player actions, define a set $\mathcal{A}_i$ for each player $i \in I$ and call $\mathcal{A} := \mathcal{A}_1 \times \dots \times \mathcal{A}_N$ the action set. The reward function $r$ is defined on this new action set and we assume that it can change from player to player (which translates into players with different objectives).

Finally we define this model as a tuple of
- Finite set of players $I = \{ 1, \dots, N \}$
- Finite set of states $\mathcal{S}$
- For each player $i \in I$:
	- Finite set of actions $\mathcal{A}_i$
	- Reward function $r_i: \mathcal{S} \times \mathcal{A} \to \mathbb{R}$ where $\mathcal{A} := \mathcal{A}_1 \times \dots \times \mathcal{A}_N$
- State transition probability function $\{ p(\cdot \mid s, a) \mid s \in \mathcal{S}, a \in \mathcal{A} \}$ defined on a transition kernel $p(\cdot \mid s, a)$ over $\mathcal{S}$.

This model takes the name of **Stochastic game**, Shapley game, or even Markov game (even though Markov had nothing to do with it).